# 신경망 학습

- 학습: 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것
- 손실 함수: 신경망이 학습할 수 있도록 해주는 지표

## 목표

손실 함수의 결과값을 가장 작게 만드는 가중치 매개변수를 찾아야 한다.

## 데이터

- 특징(feature): 입력 데이터에서 본질적인 데이터를 정확하게 추출할 수 있도록 설계된 변환기

이미지의 특징은 보통 벡터로 기술하고, 컴퓨터 비전 분야에서는 SIFT, SURF, HOG 등의 특징을 많이 사용한다.

데이터로부터 규칙을 찾아내는 역할은 '기계'가 담당하지만, 여전히 이미지를 벡터로 변환할 때 사용하는 특징은 '사람'이 설계한다. 즉, 문제에 적합한 특징을 쓰지 않으면 좋은 결과를 얻을 수 없다.

손글씨를 파악하는 문제는 3가지로 볼 수 있다.
- 5를 쓴 이미지 => 개발자가 구현한 알고리즘 => 5!
- 5를 쓴 이미지 => 사람이 생각한 특징(SIFT, SURF, HOG) => 기계학습(SVM, KNN, ...) => 5!
- 5를 쓴 이미지 => 딥러닝 => 5!
즉, 딥러닝은 이미지에 포함된 중요한 특징까지도 '기계'가 스스로 학습한다.

### 훈련 데이터와 시험 데이터

최적의 매개변수를 찾기 위해 필요한 데이터가 훈련 데이터라면, 학습된 모델이 제대로 동작하는지 확인하기 위해서는 시험 데이터가 필요하다. 한 데이터셋에만 지나치게 최적화된 상태를 오버피팅이라고 한다.

## 손실 함수

손실 함수는 신경망 학습에서 사용하는 지표이다. 신경망 성능의 '나쁨'을 나타내며, 현재의 신경망이 훈련 데이터를 얼마나 잘 처리하지 '못'하느냐를 나타낸다. 손실 함수는 임의의 함수를 사용할 수도 있지만 일반적으로는 **평균 제곱 오차**와 **교차 엔트로피 오차**를 사용한다.

### 평균 제곱 오차
> **M**ean **S**quared **E**rror

$$
E = \frac{1}{2}\sum_k(y_k - t_k)^2
$$

- $y_k$: 신경망의 출력 (신경망이 추정한 값)
- $t_k$: 정답 레이블
- $k$: 데이터의 차원 수


In [17]:
import numpy as np

def mean_squared_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)

# 손글씨 인식이라고 가정하면
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # 소프트맥스 함수의 출력
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 답은 2일 때

print(mean_squared_error(np.array(y), np.array(t))) # 0.09750000000000003

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] # 7일 확률이 제일 높다고 잘못 판단할 때
print(mean_squared_error(np.array(y), np.array(t))) # 0.5975

0.09750000000000003
0.5975


2라고 제대로 판단했을 때에는 손실 함수의 결과가 0.0975지만 7이라고 잘못 판단할 경우 0.5975로 손실 함수의 결과값이 높아진다. 즉, 첫번째 결과를 도출한 학습이 더 잘됐다.

### 교차 엔트로피 오차
> **C**ross **E**ntropy **E**rror

$$
E = - \sum_kt_k\log y_k
$$

- $y_k$: 신경망의 출력
- $t_k$: 정답 레이블 (정답에 해당하는 인덱스만 1)

In [18]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 답은 2일 때

print(cross_entropy_error(np.array(y), np.array(t))) # 0.510825457099338

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] # 7일 확률이 제일 높다고 잘못 판단할 때

print(cross_entropy_error(np.array(y), np.array(t))) # 2.302584092994546

0.510825457099338
2.302584092994546


MSE와 마찬가지로 잘못 판단했을 때에는 손실 함수의 결과값이 증가하는 것을 볼 수 있다.

### 미니배치 학습